In [32]:
import numpy as np
import cv2
from sklearn.linear_model import RANSACRegressor
from itertools import groupby
import math
lr_img = cv2.imread('R:/A_1_2934.png')
hr_img = cv2.imread('R:/A_0_2934.png')

MAX_FEATURES = 500

# Remove outliers from points
def ransac(pnt1, pnt2):
    pnt1x, pnt1y = pnt1.reshape(-1,2)[:,0].reshape(-1,1), pnt1.reshape(-1,2)[:,1].reshape(-1,1)
    pnt2x, pnt2y = pnt2.reshape(-1,2)[:,0].reshape(-1,1), pnt2.reshape(-1,2)[:,1].reshape(-1,1)
    ransacx = RANSACRegressor().fit(pnt1x, pnt2x)
    ransacy = RANSACRegressor().fit(pnt1y, pnt2y)
    inlier_maskx = ransacx.inlier_mask_
    inlier_masky = ransacy.inlier_mask_
    inliers = inlier_maskx*inlier_masky
    pnt1, pnt2 = pnt1[inliers], pnt2[inliers]
    return pnt1, pnt2

# Automatic point finding with SIFT
def auto_points(im1, im2):

    im1y, im1x, _ = im1.shape
    im2y, im2x, _ = im2.shape

    im1 = cv2.resize(im1,(max(im1x,im2x),max(im1y,im2y)),interpolation=cv2.INTER_CUBIC)
    im2 = cv2.resize(im2,(max(im1x,im2x),max(im1y,im2y)),interpolation=cv2.INTER_CUBIC)

    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
    
    sift = cv2.SIFT_create(MAX_FEATURES)
    keypoints1, descriptors1 = sift.detectAndCompute(im1Gray, None)
    keypoints2, descriptors2 = sift.detectAndCompute(im2Gray, None)
    
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    matches = bf.knnMatch(descriptors1,descriptors2,k=2)
    
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)
    
    if len(good) > 5:
        points1 = np.float32([ keypoints1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        points2 = np.float32([ keypoints2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    
    points1[:,0,0], points1[:,0,1] = points1[:,0,0]*im1x/max(im1x,im2x), points1[:,0,1]*im1y/max(im1y,im2y)
    points2[:,0,0], points2[:,0,1] = points2[:,0,0]*im2x/max(im1x,im2x), points2[:,0,1]*im2y/max(im1y,im2y)

    points1, points2 = ransac(points1, points2)
    _, ind1 = np.unique(points1, axis=0, return_index=True)
    _, ind2 = np.unique(points2, axis=0, return_index=True)
    remrows = np.intersect1d(ind1, ind2)
    points1, points2 = points1[remrows], points2[remrows]
    
    return points1, points2

# Creates a histogram of the longest stretch of consecutive ones for every row in an array
def longest_ones(matrix):
    result = []
    for row in matrix:
        max_stretch = 0
        for _, group in groupby(row):
            if _ == 1:
                max_stretch = max(max_stretch, len(list(group)))
        result.append(max_stretch)
    return result

# Finds the bounds of the largest rectangle in a histogram
def get_largest_rectangle_indices(heights):
    stack = [-1]
    max_area = 0
    max_indices = (0, 0)
    for i in range(len(heights)):
        while stack[-1] != -1 and heights[stack[-1]] >= heights[i]:
            current_height = heights[stack.pop()]
            current_width = i - stack[-1] - 1
            current_area = current_height * current_width
            if current_area > max_area:
                max_area = current_area
                max_indices = (stack[-1] + 1, i - 1)
        stack.append(i)
    while stack[-1] != -1:
        current_height = heights[stack.pop()]
        current_width = len(heights) - stack[-1] - 1
        current_area = current_height * current_width
        if current_area > max_area:
            max_area = current_area
            max_indices = (stack[-1] + 1, len(heights) - 1)
    return max_indices

# Find a large usable rectangle from a transformed dummy array
def find_rectangle(arr):
    
    rowhist = longest_ones(arr)
    colhist = longest_ones(arr.T)
    rows = get_largest_rectangle_indices(rowhist)
    cols = get_largest_rectangle_indices(colhist)
    
    if 0 in arr[rows[0]:rows[1]+1,cols[0]:cols[1]+1]:
        while 0 in arr[rows[0]:rows[1]+1,cols[0]:cols[1]+1]:
            rows += np.array([1,-1])
            cols += np.array([1,-1])
        while cols[0] > 0 and 0 not in arr[rows[0]:rows[1]+1,cols[0]-1]:
            cols[0] -= 1
        while cols[1] < arr.shape[1]-1 and 0 not in arr[rows[0]:rows[1]+1,cols[1]+1]:
            cols[1] += 1
        while rows[0] > 0 and 0 not in arr[rows[0]-1,cols[0]:cols[1]+1]:
            rows[0] -= 1
        while rows[1] < arr.shape[0]-1 and 0 not in arr[rows[1]+1,cols[0]:cols[1]+1]:
            rows[1] += 1
    
    return np.array([rows[0], cols[0]]), np.array([rows[1], cols[1]])


In [37]:
points1, points2 = auto_points(hr_img, lr_img)
h, _ = cv2.estimateAffine2D(points1, points2, cv2.RANSAC)
sx = math.sqrt(h[0,0]**2+h[1,0]**2)
sy = math.sqrt(h[0,1]**2+h[1,1]**2)
h

In [65]:
from data_prepare.misc import resize, ringing, FilterDict, Filter

lr_img = cv2.imread('R:/shugatenFg/A_1_2934.png')
hr_img = cv2.imread('R:/shugatenFg/A_0_2934.png')
h,w,_ = hr_img.shape
for pad_top in range(5):
    for pad_left in range(5):
        pad_bot = 5 - (h+pad_top)%5
        pad_right = 5 - (w+pad_left)%5
        padded_img = cv2.copyMakeBorder(hr_img, pad_top, pad_bot, pad_left, pad_right, cv2.BORDER_CONSTANT, value=[0, 0, 0])
        scaled_img = resize(padded_img, tuple([int(i*0.6) for i in padded_img.shape[:2][::-1]]), Filter.CV2_LINEAR)
        scaled_img = ringing(scaled_img, 1.5, 0.27, 0.67)
        # cv2.imwrite(f"R:/shugatenFg/{pad_top}-{pad_left}-60.png", scaled_img)

        points1, points2 = auto_points(lr_img, scaled_img)
        hh, _ = cv2.estimateAffine2D(points1, points2, cv2.RANSAC)
        # sx = math.sqrt(hh[0,0]**2+hh[1,0]**2)
        # sy = math.sqrt(hh[0,1]**2+hh[1,1]**2)
        print(pad_top, pad_left, hh[:,2])
# hr_img = cv2.imread('R:/shugatenFg/A_0_2934.png')
# pad_top = 4
# pad_left = 2
# pad_bot = 5 - (h+pad_top)%5
# pad_right = 5 - (w+pad_left)%5
# padded_img = cv2.copyMakeBorder(hr_img, pad_top, pad_bot, pad_left, pad_right, cv2.BORDER_CONSTANT, value=[0, 0, 0])
# scaled_img = resize(padded_img, tuple([int(i*0.6) for i in padded_img.shape[:2][::-1]]), Filter.CV2_AREA)
# cv2.imwrite(f"R:/shugatenFg/{pad_top}-{pad_left}-cv2area.png", scaled_img)

0 0 [-2.23859854 -2.14877226]
0 1 [-1.36374921 -2.22308308]
0 2 [-0.76551722 -2.17497088]
0 3 [-0.20153299 -2.29239725]
0 4 [ 0.40524882 -2.19772949]
1 0 [-2.031621 -1.542868]
1 1 [-1.17989881 -1.65812278]
1 2 [-0.73974083 -1.54782089]
1 3 [-0.139623   -1.79162259]
1 4 [ 0.34315947 -1.65519038]
2 0 [-1.93038891 -1.12155183]
2 1 [-1.21288163 -0.82189713]
2 2 [-0.69293697 -1.2278122 ]
2 3 [-0.12492002 -0.99495164]
2 4 [ 0.42018179 -0.92756185]
3 0 [-2.06988501 -0.36380147]
3 1 [-1.39089359 -0.39984571]
3 2 [-0.71428687 -0.19553937]
3 3 [-0.08950929 -0.42595218]
3 4 [ 0.45728114 -0.38579176]
4 0 [-1.96097659  0.18238352]
4 1 [-1.52810911 -0.0135974 ]
4 2 [-0.67339864  0.1299934 ]
4 3 [-0.0497878   0.18450905]
4 4 [0.46244831 0.18500287]


In [74]:
hr_img = cv2.imdecode(np.fromfile('R:/shugatenFg/0/氷織Ａ_2934.png', dtype=np.uint8), cv2.IMREAD_UNCHANGED)
pad_top = 0
pad_left = 0
pad_bot = 5 - (h+pad_top)%5
pad_right = 5 - (w+pad_left)%5
padded_img = cv2.copyMakeBorder(hr_img, pad_top, pad_bot, pad_left, pad_right, cv2.BORDER_CONSTANT, value=[0, 0, 0])
scaled_img1 = resize(padded_img, tuple([int(i*0.6) for i in padded_img.shape[:2][::-1]]), Filter.CV2_LINEAR)
scaled_img2 = resize(padded_img, tuple([int(i*0.6) for i in padded_img.shape[:2][::-1]]), Filter.CATROM)
scaled_img = (scaled_img1.astype(float) + scaled_img2.astype(float)) / 2
cv2.imwrite(f"R:/{pad_top}-{pad_left}-cv2linear_catrom.png", scaled_img)

True

In [10]:
import numpy as np
import cv2
from data_prepare.misc import resize, ringing, FilterDict, Filter

hr_img = cv2.imdecode(np.fromfile('R:/shugatenFg/0_50/ショコラＡ_136.png', dtype=np.uint8), cv2.IMREAD_UNCHANGED)
scaled_img= resize(hr_img, tuple([int(i*0.72) for i in hr_img.shape[:2][::-1]]), Filter.CV2_CUBIC)
cv2.imwrite(f"R:/CV2_CUBIC.png", scaled_img)

True